In [ ]:
# Import Statements

%matplotlib inline
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torchsample
from torchsample import transforms as ts_transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
from tensorboard import SummaryWriter
from datetime import datetime
import importlib

from torchsample.transforms import RangeNorm

import functions.fine_tune as ft

plt.ion()   # interactive mode

## 1 - Load Data

Before running the code, the data should be downloaded and foldered in the way that is usable for imagefolder function of the PyTorch. The following code assumes that the main directory for the dataset is 'data_dir' and it includes subdirectories for all of the separate classes.

For details on how to create those folders, pleaserefer to 'dataset/Folder_images.ipynb'

In [ ]:
'''
Macros
'''
#uniform_sampler=False
batch_size=64
split=1000
random_seed=1
shuffle=True

# Data augmentation and normalization for training 
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


'''Create ImageFolder objects for training and test'''
data_dir = '../images/0to5'
#data_dir = '/home/mtezcan/Documents/amazon/bin_images/0to5'
dset_train = datasets.ImageFolder(data_dir, data_transforms['train'])
dset_val = datasets.ImageFolder(data_dir, data_transforms['val'])

'''Validation split'''
num_train = len(dset_train)
indices = list(range(num_train))
val_idx=np.loadtxt('./dataset/validation.txt').astype(np.int)
train_idx=np.setdiff1d(indices,val_idx)

if shuffle == True:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]

'''Sampler functions for validation and training'''
sampler_train = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(val_idx)

'''Define dataset loaders'''
dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,sampler=sampler_train,
                                                    num_workers=12),
                'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,sampler=sampler_val,
                                                    num_workers=12)}


dset_sizes={'train':len(dset_train)-1000,'val':1000}
dset_classes = dset_train.classes
use_gpu = torch.cuda.is_available()

if use_gpu:
    print('GPU is available')
else:
    print('!!!!! NO CUDA GPUS DETECTED')

Let's visualize a few training images so as to understand the data
augmentations.



In [ ]:
inputs, classes = next(iter(dset_loaders['val']))
out = torchvision.utils.make_grid(inputs,nrow=8)
print('Size of the input tensors in one batch after grid is  '+str(out.size()))
plt.figure(figsize=(12,12))
ft.imshow(out, title=[dset_classes[x] for x in classes])

## 2- Log Keeping

This section includes the functions defined for the log keeping. Since CNNs require lots of trials, I found it easy to record the properties of the each trial with their performances in an excel file. I also added tnesorboard summaries for every trial and individual text files for showing the details of the network.

Below code declares the required parameters for the network which will be also used inside log keeping

In [ ]:
network='resnet34' #Initial network archtiecture.'loaded'forusing a saved network
networkName='resnet18_real_sgd_multisoft_August29  19:06:27' #Directory for the saved network
optimizer='sgd' #Optimizer function
iter_loc=14 #Number of the first column in the excel file for writing the results.
end_to_end=True #Booolean to decide whether to train the network end-to-end or not.
lr=0.01 #Initial learning rate
momentum=0.9
weight_decay=0.0005
lr_scheduler=ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch=10 #Number of epoch for learning rate decay
pretrained=True 
mse_loss=False #Scalar MSE loss
nclasses=6 #Number of output classes

'''Multipliers for loss functions'''
single_loss=1.
multi_loss=0.

comment=' ' #Additional comments if any

Let us define the functions for creating a text file and adding networkproperties to an excel file. 

In [ ]:
def writeLog(logname):
    '''
    Creates a text file named Network_properties.txt inside runs/'logname'
    '''
    f=open('runs/'+logname+'/Network_properties.txt','w')
    f.write('Batch size: '+str(batch_size)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Random seed: '+str(random_seed)+'\n')
    f.write('Shuffle: '+str(shuffle)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Network: '+network+'\n')
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    f.write('Criterion: '+crt+'\n')
    f.write('Learning rate: '+str(lr)+'\n')
    f.write('Momentum: '+str(momentum)+'\n')
    f.write('Leraning Rate Scheduler: '+str(lr_scheduler)+'\n')
    f.write('Leraning Rate Decay Period: '+str(lr_decay_epoch)+'\n')
    f.write('Network is pretrained: '+str(pretrained)+'\n')
    f.write('Network laoded from: '+networkName+'\n')
    f.write('MSE loss function: '+str(mse_loss)+'\n')
    f.close()

In [ ]:
import openpyxl
import time

def writeLog_xlsx(logname='logs.xlsx',iter_loc=14):
    '''
    Adds a line to logs.xlsx with the network properties and outcomes.
    :param iter_loc: First column to record the outcomes.
    '''
    book = openpyxl.load_workbook(logname)
    sheet = book.active
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    if network=='loaded':
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'),networkName,str(split),str(random_seed),str(shuffle),
               optimizer, crt,str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),str(pretrained),
               str(batch_size))
    else:
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'),network,str(split),str(random_seed),str(shuffle),
               optimizer, crt,str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),str(pretrained),
               str(batch_size))
    sheet.append(specs)
    current_row = sheet.max_row
    sheet.cell(row=current_row, column=iter_loc+5).value = comment
    book.save(logname)

## 3 - Training and Validation

In this part we will define the functions for training a CNN with different properties and loss functions.

The following function takes bunch of properties defined in the beginning of Section-2 as input and creates network using those properties.

In [ ]:
def network_loader(comment=comment,
                    network=network,
                    networkName=networkName,
                    optimizer=optimizer,
                    iter_loc=iter_loc,
                    end_to_end=end_to_end,
                    lr=lr,
                    momentum=momentum,
                    weight_decay=weight_decay,
                    lr_scheduler=lr_scheduler,
                    lr_decay_epoch=lr_decay_epoch,
                    pretrained=pretrained,
                    mse_loss=mse_loss,
                    nclasses=nclasses):
    
    '''Load the network from pytorch'''
    if(network=='resnet18'):
        model_ft = models.resnet18(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features #Change the last layer to adapt new classes
        if(mse_loss): #For MSE loss last layer should be a scalar
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet34'):
        model_ft = models.resnet34(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet50'):
        model_ft = models.resnet50(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet101'):
        model_ft = models.resnet101(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='alexnet'):
        model_ft = models.alexnet(pretrained=pretrained)
        num_ftrs = model_ft.classifier[6].out_features
        setattr(model_ft.classifier, '7', nn.ReLU(inplace=True))
        setattr(model_ft.classifier, '8', nn.Dropout())
        setattr(model_ft.classifier, '9', nn.Linear(num_ftrs,nclasses))

    elif(network=='loaded'):#Use the saved network when network='loaded'
        model_ft = torch.load('./saved_models/'+networkName)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, nclasses)
    else:
        raise ValueError('Undefined network '+network)

    if use_gpu:
        model_ft = model_ft.cuda()

    '''Define the optimizer function'''
    if(optimizer=='adam'):
        optimizer_ft = optim.Adam(model_ft.parameters(),lr=lr,weight_decay=weight_decay)
    elif(optimizer=='sgd'):
        if(end_to_end):
            optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=momentum)
        else:
            optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=lr, momentum=momentum,weight_decay=weight_decay)
    return model_ft, optimizer_ft

Now define a simple function to be able to run our training in a single line

In [ ]:
importlib.reload(ft)

def run_network():
    '''
    Cretaes the log files and starts the training
    '''
    model_ft, optimizer_ft = network_loader(comment=comment, #'Tested for three rooms'
                                            network=network,
                                            networkName=networkName,
                                            optimizer=optimizer,
                                            iter_loc=iter_loc,
                                            end_to_end=end_to_end,
                                            lr=lr,
                                            momentum=momentum,
                                            weight_decay=weight_decay,
                                            lr_scheduler=lr_scheduler,
                                            lr_decay_epoch=lr_decay_epoch,
                                            pretrained=pretrained,
                                            mse_loss=mse_loss,
                                            nclasses=nclasses)
    
    '''Name of the trial'''
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    logname=network+'_'+'_'+optimizer+'_'+crt+'_'+datetime.now().strftime('%B%d  %H:%M:%S')
    writer = SummaryWriter('runs/'+logname) #For tensorboard
    writeLog(logname)
    writeLog_xlsx()
    
    '''Start trianing'''
    best_model, last_model = ft.train_model(model_ft,optimizer_ft, lr_scheduler,dset_loaders,
                            dset_sizes,writer,use_gpu=use_gpu,num_epochs=30,batch_size=batch_size,num_log=250,
                            multi_prob=False,lr_decay_epoch=lr_decay_epoch,init_lr=lr,mse_loss=mse_loss,
                            iter_loc=iter_loc,cross_loss=single_loss,multi_loss=multi_loss)
    
    '''Save the models'''
    torch.save(best_model,'./saved_models/'+logname+'_best')
    torch.save(last_model,'./saved_models/'+logname+'_last')
    
    '''Frre up the memory'''
    del model_ft

## 4 - Trials

The loss function we used in the experiments is the following,

$$loss(\mathbf{y},\hat{y})=(1-\lambda )loss_{single}(\mathbf{y},\hat{y})+\lambda loss_{multi}(\mathbf{y},\hat{y})$$
where $\mathbf{y}$ is the ground truth and $\hat{y}$ is the prediction.

Now lets test our function for different values of $\lambda$.

In [ ]:
mse_loss=False
for lmbda in [0., 0.7, 1.]:
    single_loss=1.-lmbda
    multi_loss = lmbda
run_network()